# GPU acceleration

In [1]:
import time
from functools import wraps

import numpy as np

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Function '{func.__name__}' executed in {end - start:.4f} seconds")
        return result

    return wrapper

In [3]:
try:
    import google.colab

    IN_COLAB = True
    !pip install git+https://github.com/mark-hobbs/pypd.git
    print("Package installed successfully")
    import pypd
except ImportError:
    IN_COLAB = False
    import pypd

    print("Not running in Colab")

Not running in Colab


In [4]:
if IN_COLAB:
    try:
        gpu_info = !nvidia-smi
        gpu_info = "\n".join(gpu_info)
        print("GPU Information:")
        print(gpu_info)
    except:
        print("GPU information not available")

    try:
        import multiprocessing

        cpu_info = f"Number of CPU cores: {multiprocessing.cpu_count()}"
        print("\nCPU Information:")
        print(cpu_info)
    except:
        print("CPU information not available")
else:
    print("Not running in Colab.")

Not running in Colab.


## Build model

Crack branching

In [5]:
def build_particle_coordinates(dx, n_div_x, n_div_y):
    particle_coordinates = np.zeros([n_div_x * n_div_y, 2])
    counter = 0

    for i_y in range(n_div_y):  # depth
        for i_x in range(n_div_x):  # length
            coord_x = dx * i_x
            coord_y = dx * i_y
            particle_coordinates[counter, 0] = coord_x
            particle_coordinates[counter, 1] = coord_y
            counter += 1

    return particle_coordinates

In [6]:
def build_boundary_conditions(particles, dx):
    bc_flag = np.zeros((len(particles), 2), dtype=np.intc)
    bc_unit_vector = np.zeros((len(particles), 2), dtype=np.intc)

    tol = 1e-6

    for i, particle in enumerate(particles):
        if particle[1] < (0.02 + tol):
            bc_flag[i, 1] = 1
            bc_unit_vector[i, 1] = -1
        if particle[1] > (0.18 - dx - tol):
            bc_flag[i, 1] = 1
            bc_unit_vector[i, 1] = 1

    return bc_flag, bc_unit_vector

In [7]:
dx = 1e-3
n_div_x = np.rint(0.4 / dx).astype(int)
n_div_y = np.rint(0.2 / dx).astype(int)
notch = [np.array([0 - dx, 0.1 - (dx / 2)]), np.array([0.2, 0.1 - (dx / 2)])]

In [8]:
x = build_particle_coordinates(dx, n_div_x, n_div_y)
flag, unit_vector = build_boundary_conditions(x, dx)

In [9]:
material = pypd.Material(name="homalite", E=4.55e9, Gf=38.46, density=1230, ft=2.5)
bc = pypd.BoundaryConditions(flag, unit_vector, magnitude=1e-4)
particles = pypd.ParticleSet(x, dx, bc, material)
bonds = pypd.BondSet(particles, influence=pypd.Constant, notch=notch)
model = pypd.Model(particles, bonds)

/Users/markhobbs/Documents/02-repositories/pypd/pypd/tools.py:93: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = alpha_numerator / denominator
/Users/markhobbs/Documents/02-repositories/pypd/pypd/tools.py:94: RuntimeWarning: divide by zero encountered in scalar divide
  beta = beta_numerator / denominator


## Speed testing

In [10]:
@timeit
def compute_nodal_forces_a(
    x,
    u,
    cell_volume,
    bondlist,
    d,
    c,
    f_x,
    f_y,
    material_law,
    surface_correction_factors,
):
    """
    Compute particle forces - employs bondlist

    Parameters
    ----------
    bondlist : ndarray (int)
        Array of pairwise interactions (bond list)

    x : ndarray (float)
        Material point coordinates in the reference configuration

    u : ndarray (float)
        Nodal displacement

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed

    c : float
        Bond stiffness

    material_law : function

    Returns
    -------
    node_force : ndarray (float)
        Nodal force array

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed
    """

    n_nodes = np.shape(x)[0]
    n_dimensions = np.shape(x)[1]
    n_bonds = np.shape(bondlist)[0]
    node_force = np.zeros((n_nodes, n_dimensions))

    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        xi_x = x[node_j, 0] - x[node_i, 0]
        xi_y = x[node_j, 1] - x[node_i, 1]

        xi_eta_x = xi_x + (u[node_j, 0] - u[node_i, 0])
        xi_eta_y = xi_y + (u[node_j, 1] - u[node_i, 1])

        xi = np.sqrt(xi_x**2 + xi_y**2)
        y = np.sqrt(xi_eta_x**2 + xi_eta_y**2)
        stretch = (y - xi) / xi

        d[k_bond] = material_law(k_bond, stretch, d[k_bond])

        f = (
            stretch
            * c[k_bond]
            * (1 - d[k_bond])
            * cell_volume
            * surface_correction_factors[k_bond]
        )
        f_x[k_bond] = f * xi_eta_x / y
        f_y[k_bond] = f * xi_eta_y / y

    # Reduce bond forces to particle forces
    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        node_force[node_i, 0] += f_x[k_bond]
        node_force[node_j, 0] -= f_x[k_bond]
        node_force[node_i, 1] += f_y[k_bond]
        node_force[node_j, 1] -= f_y[k_bond]

    return node_force, d

In [11]:
from numba import njit, prange

@timeit
@njit(parallel=True, fastmath=True)
def compute_nodal_forces_b(
    x,
    u,
    cell_volume,
    bondlist,
    d,
    c,
    f_x,
    f_y,
    material_law,
    surface_correction_factors,
):
    """
    Compute particle forces - employs bondlist

    Parameters
    ----------
    bondlist : ndarray (int)
        Array of pairwise interactions (bond list)

    x : ndarray (float)
        Material point coordinates in the reference configuration

    u : ndarray (float)
        Nodal displacement

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed

    c : float
        Bond stiffness

    material_law : function

    Returns
    -------
    node_force : ndarray (float)
        Nodal force array

    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed
    """

    n_nodes = np.shape(x)[0]
    n_dimensions = np.shape(x)[1]
    n_bonds = np.shape(bondlist)[0]
    node_force = np.zeros((n_nodes, n_dimensions))

    for k_bond in prange(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        xi_x = x[node_j, 0] - x[node_i, 0]
        xi_y = x[node_j, 1] - x[node_i, 1]

        xi_eta_x = xi_x + (u[node_j, 0] - u[node_i, 0])
        xi_eta_y = xi_y + (u[node_j, 1] - u[node_i, 1])

        xi = np.sqrt(xi_x**2 + xi_y**2)
        y = np.sqrt(xi_eta_x**2 + xi_eta_y**2)
        stretch = (y - xi) / xi

        d[k_bond] = material_law(k_bond, stretch, d[k_bond])

        f = (
            stretch
            * c[k_bond]
            * (1 - d[k_bond])
            * cell_volume
            * surface_correction_factors[k_bond]
        )
        f_x[k_bond] = f * xi_eta_x / y
        f_y[k_bond] = f * xi_eta_y / y

    # Reduce bond forces to particle forces
    for k_bond in range(n_bonds):
        node_i = bondlist[k_bond, 0]
        node_j = bondlist[k_bond, 1]

        node_force[node_i, 0] += f_x[k_bond]
        node_force[node_j, 0] -= f_x[k_bond]
        node_force[node_i, 1] += f_y[k_bond]
        node_force[node_j, 1] -= f_y[k_bond]

    return node_force, d

In [17]:
compute_nodal_forces_a(particles.x, 
                       particles.u, 
                       particles.cell_volume,
                       bonds.bondlist, 
                       bonds.d, 
                       bonds.c, 
                       bonds.f_x,
                       bonds.f_y,
                       bonds.constitutive_law.calculate_bond_damage, 
                       bonds.surface_correction_factors);

compute_nodal_forces_b(particles.x, 
                       particles.u, 
                       particles.cell_volume,
                       bonds.bondlist, 
                       bonds.d, 
                       bonds.c, 
                       bonds.f_x,
                       bonds.f_y,
                       bonds.constitutive_law.calculate_bond_damage, 
                       bonds.surface_correction_factors);

Function 'compute_nodal_forces_a' executed in 3.8570 seconds
Function 'compute_nodal_forces_b' executed in 0.0052 seconds


## Numba CUDA

In [13]:
from numba import cuda

print(cuda.is_available())

False


In [14]:
def get_cuda_device_info(verbose=True):
    """
    Retrieve comprehensive information about the current CUDA device.
    
    Parameters:
    -----------
    verbose : bool, optional
        If True, print device information. If False, return as dictionary.
    
    Returns:
    --------
    dict or None
        Dictionary of device properties if verbose=False, otherwise None
    """
    try:
        # Get current device
        device = cuda.get_current_device()
        context = cuda.current_context()
        
        # Collect device information
        device_info = {
            "name": device.name,
            "compute_capability": device.compute_capability,
            
            # Memory information
            "total_memory_gb": context.get_memory_info().total / 1e9,
            "free_memory_gb": context.get_memory_info().free / 1e9,
            
            # Compute resources
            "multiprocessors": device.MULTIPROCESSOR_COUNT,
            "max_threads_per_block": device.MAX_THREADS_PER_BLOCK,
            
            # Grid and block limitations
            "max_grid_dimensions": {
                "x": device.MAX_GRID_DIM_X,
                "y": device.MAX_GRID_DIM_Y,
                "z": device.MAX_GRID_DIM_Z
            },
            
            # Detailed performance characteristics
            "warp_size": device.WARP_SIZE,
            "clock_rate_khz": device.CLOCK_RATE,
            "memory_clock_rate_khz": device.MEMORY_CLOCK_RATE,
        }
        
        # Formatted printing if verbose
        if verbose:
            print("CUDA Device Information:")
            print("-" * 40)
            print(f"{'Device Name:':<25} {device_info['name']}")
            print(f"{'Compute Capability:':<25} {device_info['compute_capability']}")
            
            print("\nMemory:")
            print(f"{'Total Memory:':<25} {device_info['total_memory_gb']:.2f} GB")
            print(f"{'Free Memory:':<25} {device_info['free_memory_gb']:.2f} GB")
            
            print("\nCompute Resources:")
            print(f"{'Multiprocessors:':<25} {device_info['multiprocessors']}")
            print(f"{'Max Threads per Block:':<25} {device_info['max_threads_per_block']}")
            
            print("\nGrid Limitations:")
            print(f"{'Max Grid Dimensions X:':<25} {device_info['max_grid_dimensions']['x']}")
            print(f"{'Max Grid Dimensions Y:':<25} {device_info['max_grid_dimensions']['y']}")
            print(f"{'Max Grid Dimensions Z:':<25} {device_info['max_grid_dimensions']['z']}")
            
            print("\nAdditional Characteristics:")
            print(f"{'Warp Size:':<25} {device_info['warp_size']}")
            print(f"{'Clock Rate:':<25} {device_info['clock_rate_khz']/1e6:.2f} GHz")
            print(f"{'Memory Clock Rate:':<25} {device_info['memory_clock_rate_khz']/1e6:.2f} GHz")
            
            print("\nEstimated Parallel Computation Capacity:")
            max_blocks_per_mp = device_info['max_grid_dimensions']['x']
            total_max_blocks = max_blocks_per_mp * device_info['multiprocessors']
            print(f"{'Max Blocks per Multiprocessor:':<25} {max_blocks_per_mp}")
            print(f"{'Total Potential Parallel Blocks:':<25} {total_max_blocks}")
        
        return device_info if not verbose else None
    
    except Exception as e:
        print(f"Error retrieving CUDA device information: {e}")
        return None

def estimate_optimal_launch_configuration(total_work_items, max_threads_per_block=None):
    """
    Estimate optimal CUDA kernel launch configuration
    
    Parameters:
    -----------
    total_work_items : int
        Total number of work items to process
    max_threads_per_block : int, optional
        Maximum threads per block (defaults to device maximum)
    
    Returns:
    --------
    tuple
        (threads_per_block, blocks_per_grid)
    """
    # Get device info if not provided
    device = cuda.get_current_device()
    
    # Use device max if not specified
    if max_threads_per_block is None:
        max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    
    # Standard warp size (typically 32)
    warp_size = device.WARP_SIZE
    
    # Round threads per block to nearest multiple of warp size
    threads_per_block = min(
        max_threads_per_block, 
        ((total_work_items + warp_size - 1) // warp_size) * warp_size
    )
    
    # Calculate blocks, ensuring we don't exceed grid dimension limits
    blocks_per_grid = (total_work_items + threads_per_block - 1) // threads_per_block
    
    # Respect max grid dimension
    blocks_per_grid = min(blocks_per_grid, device.MAX_GRID_DIM_X)
    
    return threads_per_block, blocks_per_grid

In [15]:
get_cuda_device_info()

print("\nLaunch Configuration Estimation:")
total_work_items = 1_000_000
threads, blocks = estimate_optimal_launch_configuration(total_work_items)
print(f"For {total_work_items} work items:")
print(f"Threads per Block: {threads}")
print(f"Blocks per Grid: {blocks}")

Error retrieving CUDA device information: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

Launch Configuration Estimation:


CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: